In [10]:
import pandas as pd
from SVM import *
import cvxpy as cp

**Loading Datasets**

In [44]:
X = np.matrix(pd.read_csv("/Users/ryanlucas/Desktop/df_X.csv")).T
Y = np.array(pd.read_csv("/Users/ryanlucas/Desktop/df_Y.csv")).reshape(-1)

In [109]:
def ERM_SVM(ξ):

    model = pic.Problem() # Model instantiation

    # This part here will depend on the type of the problem. (nature of ξ and θ)------
    X, Y = ξ[0], ξ[1]
    N, d = X.shape

    # Decision Variables
    θ = pic.RealVariable("theta", (d+1, 1))
    loss = pic.RealVariable("loss", (N, 1))  # Models \loss^\epsilon (i)

    # Objective Function
    model.set_objective('min', 1/N*pic.sum(loss))

    # Loss definition-------------------------
    # SVM loss = max{1-Y'(w'X-b)}, loss_epsilon = max{1-Y'(w'X-b) + ϵ |w|_2^2/|w|}
    for i in range(0, N):
        model.add_constraint(loss[i] >= 0)
        model.add_constraint(loss[i] >= 1 - Y[i]*(θ[0:d].T*X[i, :] - θ[-1]))
    # ----------------------------------------

    model.solve(solver='mosek')

    return np.array(θ.value), model.value


In [121]:
def ERM_SVM(ξ):

    X, Y = ξ[0], ξ[1]
    d, N = X.shape

    θ = cp.Variable(shape = (d+1, 1))
    loss = cp.Variable(shape = (N, 1)) # Models \loss^\epsilon (i)

    objective = cp.Minimize(1/N * cp.sum(loss))

    nonnegativity_constraints = []
    loss_constraints = []

    for i in range(0, N):
        nonnegativity_constraints.append(loss[i] >= 0)
        loss_constraints.append(loss[i] >= 1 - Y[i]*(θ[0:d].T@X[:, i] - θ[-1]))

    complete_constraints = nonnegativity_constraints + loss_constraints

    model = cp.Problem(
                    objective=objective,
                    constraints=complete_constraints
                    )

    model.solve()

    return θ.value, model.value